In [1]:
!pip install -q BlackBoxAuditing 
!pip install -q aif360

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 5.1 MB/s eta 0:00:0000:01


In [2]:
from sklearn.datasets import fetch_openml
#from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score,precision_score,recall_score
import seaborn as sns
#from fairlearn.metrics import selection_rate
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from aif360.datasets import BinaryLabelDataset
from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing import DisparateImpactRemover
from sklearn.metrics import accuracy_score, confusion_matrix
from aif360.metrics import ClassificationMetric
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dense, Dropout, Input
from tqdm import tqdm




# import numpy as np
# import pandas as pd
# import tensorflow as tf
# tf.compat.v1.enable_eager_execution()
# import os
# import cv2
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split, StratifiedKFold
# from sklearn.metrics import accuracy_score
# import keras
# from keras.layers import *
# from keras.models import *
# from keras.optimizers import Adam
# from aif360.datasets import StandardDataset
# from aif360.metrics import ClassificationMetric

# import warnings
# warnings.simplefilter(action='ignore', category='PerformanceWarning')


In [4]:
df = pd.read_csv('/kaggle/input/loan-prediction-practice-av-competition/train_csv.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

# Preparing data

# Filling missing values

In [6]:
df.Gender.fillna(df.Gender.mode().values[0], inplace = True)

df.Married.fillna(df.Married.mode().values[0], inplace=True)

df.Dependents.fillna(df.Dependents.mode().values[0], inplace=True)

df.Self_Employed.fillna(df.Self_Employed.mode().values[0], inplace=True)

df.LoanAmount.fillna(round(df.LoanAmount.mean(),0),inplace=True)

df.Loan_Amount_Term.fillna(round(df.Loan_Amount_Term.mean(),0), inplace=True)

df.Credit_History.fillna(df.Credit_History.mode().values[0], inplace=True)


<ipython-input-6-f3e89564a366>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.Gender.fillna(df.Gender.mode().values[0], inplace = True)
<ipython-input-6-f3e89564a366>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

In [7]:
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# Dropping ID AND Replacing 3+ enteries

In [8]:
### dropping ID

# Drop Loan_ID
df = df.drop(['Loan_ID'], axis = 1)

### Replacing 3+

### Replacing 3+ in 'Dependents' with 3
df = df.replace({'Dependents': '3+'}, 3)

# Converting Categorical Variables into Numbers  

In [9]:

label_encoder=LabelEncoder()

df['Gender']=label_encoder.fit_transform(df['Gender'])
df['Married']=label_encoder.fit_transform(df['Married'])
df['Education']=label_encoder.fit_transform(df['Education'])
df['Self_Employed']=label_encoder.fit_transform(df['Self_Employed'])
df['Property_Area']=label_encoder.fit_transform(df['Property_Area'])
df['Loan_Status']=label_encoder.fit_transform(df['Loan_Status'])
df['Gender']=label_encoder.fit_transform(df['Gender'])
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,0,0,5849,0.0,146.0,360.0,1.0,2,1
1,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1


# Addressing and Mitigating Bias with AI Fairness 360

# Identifying Bias in the model 

# Splitting data

In [10]:
X=df.drop('Loan_Status',axis=1)
y=df['Loan_Status']
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

# Original Model 

In [11]:
from catboost import CatBoostClassifier

In [12]:
model1 = CatBoostClassifier(verbose=0)
model1.fit(X_train, y_train)

In [13]:
y_pred_test = model1.predict(X_test)
y_pred = model1.predict(X)

In [14]:
accuracy_score(y_test.values, y_pred_test)

0.7886178861788617

# Metrics

In [15]:

# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Loan_Status'])


for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([_X_test, _y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = y_pred_test
#np.argmax(y_pred_test).values,axis=1
        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())
            max_di = max(max_di, metric.disparate_impact())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
print("Max Disparate Impact (Ideal value = 1):", max_di)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.1242124212421242
Max Average Odds Difference (Ideal value = 0): 0.0897782772782773
Max Equal Opportunity Difference (Ideal value = 0): 0.18055555555555558
Max Theil Index (Ideal value = 0): 0.19007812277935573
Max Generalized Entropy Index (Ideal value = 0): 0.12393474555636722
Max Disparate Impact (Ideal value = 1): 1.8363636363636364


# Mitigation Algorithms

## Preprocessing Algorithms:

### 1- Reweighing

In [51]:

    
# for age_cls in [0,1]:
#     for priv_cls in [0,1]:
#         # Convert to AIF360 format
        
    

#     # Define the groups
#     groups = [
#         {'Gender': 0},
#         {'Gender': 1}
#         ]

#         # Iterate through the groups
#     for i, privileged_group in enumerate(groups):
#             # Define unprivileged groups as all other groups
#             unprivileged_groups = groups[:i] + groups[i+1:]

aif360_dataset_train = StandardDataset(df=pd.concat([X_train, y_train], axis=1),
                                           label_name='Loan_Status', favorable_classes=[0],
                                           protected_attribute_names=['Gender'],
                                           privileged_classes=[[1]])
                                          
weights = aif360_dataset_train.instance_weights
preprocessor = Reweighing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}])
train_dataset_reweighted = preprocessor.fit_transform(aif360_dataset_train)
#metric_e = BinaryLabelDatasetMetric(train_dataset_reweighted, unprivileged_groups=[{sensitive_attribute: 0}], privileged_groups=[{sensitive_attribute: 1}])

model_weigh = CatBoostClassifier(verbose=0)
model_weigh.fit(train_dataset_reweighted.features,train_dataset_reweighted.labels.ravel() , sample_weight=weights)
#aif360_dataset_train.features  aif360_dataset_train.labels.ravel()  
y_pred_wei = model_weigh.predict(X_test)

In [53]:
accuracy_score(y_test.values, y_pred_wei)

0.7886178861788617

In [54]:
# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
#_X_test = pd.DataFrame(X_test, columns=X.columns)
#_y_test = pd.DataFrame(y_test, columns=['Loan_Status'])


for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset1 = aif360_dataset.copy()
        aif360_pred_dataset1.labels = y_pred_wei
#np.argmax(y_pred_wei).values,axis=1
        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]

#aif360_pred_dataset.labels = 

        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset1,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())
            max_di = max(max_di, metric.disparate_impact())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
print("Max Disparate Impact (Ideal value = 1):", max_di)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.1242124212421242
Max Average Odds Difference (Ideal value = 0): 0.0897782772782773
Max Equal Opportunity Difference (Ideal value = 0): 0.18055555555555558
Max Theil Index (Ideal value = 0): 0.19007812277935573
Max Generalized Entropy Index (Ideal value = 0): 0.12393474555636722
Max Disparate Impact (Ideal value = 1): 1.8363636363636364


## 2- Disparate Impact Remover

In [55]:
  
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Loan_Status'])
 

transformed_X_Train = pd.DataFrame(index=X_train.index, columns=X_train.columns)

groups = [
    {'Gender': 0},
    {'Gender': 1},
                  ]
priv_group = groups[0]
unprivileged_groups = [groups[1]]
aif360_dataset = StandardDataset(pd.concat([X_train,y_train],axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

   
DIR = DisparateImpactRemover(repair_level=1.0, sensitive_attribute="Gender")
aif360_dataset_train = DIR.fit_transform(aif360_dataset)   
X_Train_transformed = aif360_dataset_train.features
y_Train_transformed = aif360_dataset_train.labels.ravel()
            


model_DIR = CatBoostClassifier(verbose=0)
model_DIR.fit(X_Train_transformed.astype('float32'), y_Train_transformed.astype('float32'))

#aif360_dataset_train.features  aif360_dataset_train.labels.ravel() 

In [56]:
  
y_pred_DIR= model_DIR.predict(X_test)

In [57]:
accuracy_score(y_test.values, y_pred_DIR)

0.7886178861788617

In [58]:
# Initialize variables to store maximum values of each metric

max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Loan_Status'])


for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([_X_test, _y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset2 = aif360_dataset.copy()
        aif360_pred_dataset2.labels = y_pred_DIR
#np.argmax(y_pred_wei).values,axis=1
        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]

#aif360_pred_dataset.labels = 

        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset2,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())
            max_di = max(max_di, metric.disparate_impact())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
print("Max Disparate Impact (Ideal value = 1):", max_di)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.1242124212421242
Max Average Odds Difference (Ideal value = 0): 0.0897782772782773
Max Equal Opportunity Difference (Ideal value = 0): 0.18055555555555558
Max Theil Index (Ideal value = 0): 0.19007812277935573
Max Generalized Entropy Index (Ideal value = 0): 0.12393474555636722
Max Disparate Impact (Ideal value = 1): 1.8363636363636364


# In-processing Algorithms:

## 1- Adversarial Debiasing

In [59]:

#pip install --upgrade tensorflow
# import tensorflow as tf
# print(tf.__version__)


2.17.1


In [ ]:
#pip install tensorflow

In [69]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []

# for train_index, test_index in StratifiedKFold(n_splits=5).split(X.values, y.values):
#     X_Train, X_Test = X.iloc[train_index, :], X.iloc[test_index, :]
#     y_Train, y_Test = y.iloc[train_index], y.iloc[test_index]

# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Loan_Status', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[1]])

# Apply the AdversarialDebiasing algorithm
tf.reset_default_graph()
sess = tf.Session()
adv_debiasing = AdversarialDebiasing(privileged_groups=priv_group,
                                     unprivileged_groups=unprivileged_groups,
                                     scope_name='adv_debiasing',
                                     debias=True,
                                     sess=sess)
adv_debiasing.fit(aif360_train)

# Transform the training data
aif360_train_pred = adv_debiasing.predict(aif360_train)
X_Train_transformed = aif360_train_pred.features
y_Train_transformed = aif360_train_pred.labels.ravel()

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_Train_transformed, y_Train_transformed)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds

scores.append(accuracy_score(y_test.values, preds))
print(scores[-1])

sess.close()

print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

epoch 0; iter: 0; batch classifier loss: 145.443176; batch adversarial loss: 0.665809
epoch 1; iter: 0; batch classifier loss: 137.685394; batch adversarial loss: 0.596135
epoch 2; iter: 0; batch classifier loss: 174.962219; batch adversarial loss: 0.629473
epoch 3; iter: 0; batch classifier loss: 163.631897; batch adversarial loss: 0.557075
epoch 4; iter: 0; batch classifier loss: 92.050629; batch adversarial loss: 0.545518
epoch 5; iter: 0; batch classifier loss: 110.024979; batch adversarial loss: 0.545292
epoch 6; iter: 0; batch classifier loss: 131.303696; batch adversarial loss: 0.569171
epoch 7; iter: 0; batch classifier loss: 114.447327; batch adversarial loss: 0.592333
epoch 8; iter: 0; batch classifier loss: 133.088287; batch adversarial loss: 0.521312
epoch 9; iter: 0; batch classifier loss: 78.080460; batch adversarial loss: 0.558228
epoch 10; iter: 0; batch classifier loss: 59.052254; batch adversarial loss: 0.510603
epoch 11; iter: 0; batch classifier loss: 100.485924; ba

In [70]:
# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Loan_Status'])


for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([_X_test, _y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset2 = aif360_dataset.copy()
        aif360_pred_dataset2.labels =y_pred_4
        #
#np.argmax(y_pred_4.values,axis=1)


        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]

#aif360_pred_dataset.labels = 

        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset2,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())
            #max_di = max(max_di, metric.disparate_impact())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
#print("Max Disparate Impact (Ideal value = 1):", max_di)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.0
Max Average Odds Difference (Ideal value = 0): 0.0
Max Equal Opportunity Difference (Ideal value = 0): 0.0
Max Theil Index (Ideal value = 0): 0.3123746850421525
Max Generalized Entropy Index (Ideal value = 0): 0.18333333333333326


# 2 - PrejudiceRemover

In [76]:
# import pandas as pd
# import numpy as np
# from aif360.datasets import StandardDataset
# from aif360.algorithms.inprocessing import AdversarialDebiasing
# from sklearn.model_selection import StratifiedKFold
# from aif360.algorithms.inprocessing import PrejudiceRemover

# # Function to convert dataframe to AIF360 StandardDataset
# def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
#     return StandardDataset(df,
#                            label_name=label_name,
#                            favorable_classes=favorable_classes,
#                            protected_attribute_names=protected_attribute_names,
#                            privileged_classes=privileged_classes)

# y_pred_4 = pd.Series(0, index=X.index)
# groups = [
#     {'Gender': 0},
#     {'Gender': 1},
# ]
# priv_group = groups[0]
# unprivileged_groups = [groups[1]]

# scores = []

# # for train_index, test_index in StratifiedKFold(n_splits=5).split(X.values, y.values):
# #     X_Train, X_Test = X.iloc[train_index, :], X.iloc[test_index, :]
# #     y_Train, y_Test = y.iloc[train_index], y.iloc[test_index]

# # Create a combined DataFrame for X_Train and y_Train
# combined_train = pd.concat([X_train, y_train], axis=1)

# # Convert to AIF360 format
# aif360_train = convert_to_dataset(combined_train,
#                                   label_name='Loan_Status', favorable_classes=[1],
#                                   protected_attribute_names=['Gender'],
#                                   privileged_classes=[[1, i] for i in range(7)])

# # Apply the AdversarialDebiasing algorithm
# tf.reset_default_graph()
# sess = tf.Session()
# adv_debiasing = PrejudiceRemover()
# adv_debiasing.fit(aif360_train)

# # Transform the training data
# aif360_train_pred = adv_debiasing.predict(aif360_train)
# X_Train_transformed = aif360_train_pred.features
# y_Train_transformed = aif360_train_pred.labels.ravel()

# # Train your model using the debiased training data
# model = CatBoostClassifier(verbose=0)
# model.fit(X_Train_transformed, y_Train_transformed)

# # Get predictions from your model
# preds = model.predict(X_test.values)
# y_pred_4 = preds

# scores.append(accuracy_score(y_test.values, preds))
# print(scores[-1])

# sess.close()

# print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

In [72]:
  
# _X_test = pd.DataFrame(X_test, columns=X.columns)
# _y_test = pd.DataFrame(y_test, columns=['Loan_Status'])
 
# from aif360.algorithms.inprocessing import PrejudiceRemover
# transformed_X_Train = pd.DataFrame(index=X.index, columns=X_train.columns)

# groups = [
#     {'Gender': 0},
#     {'Gender': 1},
#                   ]
# priv_group = groups[0]
# unprivileged_groups = [groups[1]]
# aif360_dataset = StandardDataset(df,
#                                          label_name='Loan_Status', favorable_classes=[age_cls],
#                                          protected_attribute_names=['Gender'],
#                                          privileged_classes=[[priv_cls]])

# remover = PrejudiceRemover()
# train_data_debiased = remover.fit(aif360_dataset)



In [77]:
# # Initialize variables to store maximum values of each metric
# max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
# _X_test = pd.DataFrame(X_test, columns=X.columns)
# _y_test = pd.DataFrame(y_test, columns=['Loan_Status'])


# for age_cls in [0,1]:
#     for priv_cls in [0,1]:
#         # Convert to AIF360 format
#         aif360_dataset = StandardDataset(df=pd.concat([_X_test, _y_test], axis=1),
#                                          label_name='Loan_Status', favorable_classes=[age_cls],
#                                          protected_attribute_names=['Gender'],
#                                          privileged_classes=[[priv_cls]])

#         # Create a dataset for the predictions
#         aif360_pred_dataset2 = aif360_dataset.copy()
#         aif360_pred_dataset2.labels = y_pred_8  
#         #
# #np.argmax(y_pred_4.values,axis=1)


#         # Define the groups
#         groups = [
#             {'Gender': 0},
#             {'Gender': 1}
#         ]

# #aif360_pred_dataset.labels = 

#         # Iterate through the groups
#         for i, privileged_group in enumerate(groups):
#             # Define unprivileged groups as all other groups
#             unprivileged_groups = groups[:i] + groups[i+1:]

#             # Calculate metrics
#             metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset2,
#                                           unprivileged_groups=unprivileged_groups,
#                                           privileged_groups=[privileged_group])

#             # Update maximum values
#             max_spd = max(max_spd, metric.statistical_parity_difference())
#             max_aod = max(max_aod, metric.average_odds_difference())
#             max_eod = max(max_eod, metric.equal_opportunity_difference())
#             max_theil = max(max_theil, metric.theil_index())
#             max_gei = max(max_gei, metric.generalized_entropy_index())
#             #max_di = max(max_di, metric.disparate_impact())

# # Print maximum values
# print("Gender Bias Identification:")
# print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
# print("Max Average Odds Difference (Ideal value = 0):", max_aod)
# print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
# print("Max Theil Index (Ideal value = 0):", max_theil)
# print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
# #print("Max Disparate Impact (Ideal value = 1):", max_di)

# Postprocessing Algorithms:

## 1- Equalized Odds Postprocessing

In [80]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.algorithms.postprocessing import EqOddsPostprocessing,CalibratedEqOddsPostprocessing

from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []

# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Loan_Status', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[1]])

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_train, y_train)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds


aif360_test = StandardDataset(pd.concat([X_test, pd.DataFrame(y_test, columns=['Loan_Status'], index=X_test.index)], axis=1),
                                      label_name='Loan_Status', favorable_classes=[1],
                                      protected_attribute_names=['Gender'],
                                      privileged_classes=[[1]])
aif360_preds = StandardDataset(pd.concat([X_test,pd.DataFrame(y_pred_4, columns=['Loan_Status'], index=X_test.index)], axis=1),
                               label_name='Loan_Status', favorable_classes=[1],
                               protected_attribute_names=['Gender'],
                               privileged_classes=[[1]])

eq_odds = EqOddsPostprocessing(unprivileged_groups=[{'Gender': 1}], privileged_groups=[{'Gender': 0}], seed=42)
eq_odds = eq_odds.fit(aif360_test, aif360_preds)
transformed_preds = eq_odds.predict(aif360_preds).labels



scores.append(accuracy_score(y_test.values, transformed_preds))
print(scores[-1])


print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

0.7723577235772358
Mean: 0.7723577235772358 
STD:  0.0 



In [36]:
# from tensorflow.keras.layers import Dense, Dropout, Input
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
# from aif360.algorithms.postprocessing import EqOddsPostprocessing,CalibratedEqOddsPostprocessing
# from aif360.datasets import StandardDataset
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import accuracy_score
# import pandas as pd
# import numpy as np
# from tqdm import tqdm
# import tensorflow.keras.backend as K



# privileged_groups = [{'Gender': 1}]
# unprivileged_groups = [{'Gender': 0}]

# n_classes = 4  # Number of classes
# scores = []
# y_pred_all_classes = np.zeros((len(y), n_classes))

# for class_index in range(n_classes):
#     for train_index, test_index in tqdm(StratifiedKFold(n_splits=5).split(X, y)):
#         X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
#         y_train, y_test = y.iloc[train_index], y.iloc[test_index]
   

#         # Convert the target to binary for the current class
#         y_train_binary = (y_train == target_value).astype(int)
#         y_test_binary = (y_test == target_value).astype(int)

        
  

#         # Predict using the standard model
#         #standard_preds = dense_model.predict(X_Test).flatten()  # Flatten for binary output

#         # Convert to AIF360 format for postprocessing
#         aif360_test = StandardDataset(pd.concat([X_test[['Gender']], pd.DataFrame(y_test_binary, columns=['Loan_Status'], index=X_test.index)], axis=1),
#                                       label_name='Loan_Status', favorable_classes=[1],
#                                       protected_attribute_names=['Gender'],
#                                       privileged_classes=[[1]])
#         aif360_preds = StandardDataset(pd.concat([X_test[['Gender']],pd.DataFrame(y_test, columns=['Loan_Status'], index=X_test.index)], axis=1),
#                                        label_name='Loan_Status', favorable_classes=[1],
#                                        protected_attribute_names=['Gender'],
#                                        privileged_classes=[[1]])

#         # Apply Equalized Odds Postprocessing
#         eq_odds = CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=42)
#         eq_odds = eq_odds.fit(aif360_test, aif360_preds)
#         transformed_preds = eq_odds.predict(aif360_preds).labels

#         # Store the predictions for the current class
#         y_pred_all_classes[test_index, class_index] = transformed_preds.squeeze()

# # Convert the predictions for each class into final multi-class predictions
# y_pred_final = np.argmax(y_pred_all_classes, axis=1)
# overall_accuracy = accuracy_score(y, y_pred_final)

# print("Overall Accuracy:", overall_accuracy)


5it [00:00, 40.53it/s]
5it [00:00, 55.12it/s]
5it [00:00, 57.87it/s]
5it [00:00, 57.31it/s]

Overall Accuracy: 0.3127035830618892


In [81]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = transformed_preds

        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.033303330333033315
Max Average Odds Difference (Ideal value = 0): 0.021332833832833853
Max Equal Opportunity Difference (Ideal value = 0): 0.041666666666666685
Max Theil Index (Ideal value = 0): 0.20986455138033092
Max Generalized Entropy Index (Ideal value = 0): 0.13668883090648923


# 3- Calibrated EqOdds Postprocessing

In [82]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.algorithms.postprocessing import EqOddsPostprocessing,CalibratedEqOddsPostprocessing

from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []

# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Loan_Status', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[1]])

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_train, y_train)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds


aif360_test = StandardDataset(pd.concat([X_test, pd.DataFrame(y_test, columns=['Loan_Status'], index=X_test.index)], axis=1),
                                      label_name='Loan_Status', favorable_classes=[1],
                                      protected_attribute_names=['Gender'],
                                      privileged_classes=[[1]])
aif360_preds = StandardDataset(pd.concat([X_test,pd.DataFrame(y_pred_4, columns=['Loan_Status'], index=X_test.index)], axis=1),
                               label_name='Loan_Status', favorable_classes=[1],
                               protected_attribute_names=['Gender'],
                               privileged_classes=[[1]])

eq_odds = CalibratedEqOddsPostprocessing(unprivileged_groups=[{'Gender': 1}], privileged_groups=[{'Gender': 0}], seed=42)
eq_odds = eq_odds.fit(aif360_test, aif360_preds)
transformed_preds = eq_odds.predict(aif360_preds).labels



scores.append(accuracy_score(y_test.values, transformed_preds))
print(scores[-1])


print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

0.7967479674796748
Mean: 0.7967479674796748 
STD:  0.0 



In [83]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = transformed_preds

        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.1539153915391539
Max Average Odds Difference (Ideal value = 0): 0.12359862359862361
Max Equal Opportunity Difference (Ideal value = 0): 0.22222222222222227
Max Theil Index (Ideal value = 0): 0.19423342274448901
Max Generalized Entropy Index (Ideal value = 0): 0.12217078189300408


## Sequential Models

### RW + AD

In [87]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []

# for train_index, test_index in StratifiedKFold(n_splits=5).split(X.values, y.values):
#     X_Train, X_Test = X.iloc[train_index, :], X.iloc[test_index, :]
#     y_Train, y_Test = y.iloc[train_index], y.iloc[test_index]

# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Loan_Status', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[1]])
weights = aif360_train.instance_weights
preprocessor = Reweighing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}])
train_dataset_reweighted = preprocessor.fit_transform(aif360_train)

# Apply the AdversarialDebiasing algorithm
tf.reset_default_graph()
sess = tf.Session()
adv_debiasing = AdversarialDebiasing(privileged_groups=priv_group,
                                     unprivileged_groups=unprivileged_groups,
                                     scope_name='adv_debiasing',
                                     debias=True,
                                     sess=sess)
adv_debiasing.fit(train_dataset_reweighted)

# Transform the training data
aif360_train_pred = adv_debiasing.predict(train_dataset_reweighted)
X_Train_transformed = aif360_train_pred.features
y_Train_transformed = aif360_train_pred.labels.ravel()

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_Train_transformed, y_Train_transformed, sample_weight=weights)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds

scores.append(accuracy_score(y_test.values, preds))
print(scores[-1])

sess.close()

print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

epoch 0; iter: 0; batch classifier loss: 93.352814; batch adversarial loss: 0.700509
epoch 1; iter: 0; batch classifier loss: 97.508163; batch adversarial loss: 0.714045
epoch 2; iter: 0; batch classifier loss: 69.579880; batch adversarial loss: 0.724420
epoch 3; iter: 0; batch classifier loss: 64.951477; batch adversarial loss: 0.739781
epoch 4; iter: 0; batch classifier loss: 94.222488; batch adversarial loss: 0.711270
epoch 5; iter: 0; batch classifier loss: 69.328491; batch adversarial loss: 0.737412
epoch 6; iter: 0; batch classifier loss: 112.959793; batch adversarial loss: 0.696730
epoch 7; iter: 0; batch classifier loss: 65.329208; batch adversarial loss: 0.697672
epoch 8; iter: 0; batch classifier loss: 67.526749; batch adversarial loss: 0.721274
epoch 9; iter: 0; batch classifier loss: 89.652214; batch adversarial loss: 0.694549
epoch 10; iter: 0; batch classifier loss: 71.272614; batch adversarial loss: 0.723012
epoch 11; iter: 0; batch classifier loss: 59.078819; batch adve

In [88]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = preds

        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.00990099009900991
Max Average Odds Difference (Ideal value = 0): 0.020833333333333332
Max Equal Opportunity Difference (Ideal value = 0): 0.041666666666666664
Max Theil Index (Ideal value = 0): 0.3013248488555674
Max Generalized Entropy Index (Ideal value = 0): 0.17582417582417567


### DIR + AD

In [96]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []


# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Loan_Status', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[1]])

DIR = DisparateImpactRemover(repair_level=1.0, sensitive_attribute="Gender")
aif360_train = DIR.fit_transform(aif360_train)   

# Apply the AdversarialDebiasing algorithm
tf.reset_default_graph()
sess = tf.Session()
adv_debiasing = AdversarialDebiasing(privileged_groups=priv_group,
                                     unprivileged_groups=unprivileged_groups,
                                     scope_name='adv_debiasing',
                                     debias=True,
                                     sess=sess)
adv_debiasing.fit(aif360_train)

# Transform the training data
aif360_train_pred = adv_debiasing.predict(aif360_train)
X_Train_transformed = aif360_train_pred.features
y_Train_transformed = aif360_train_pred.labels.ravel()

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_Train_transformed, y_Train_transformed)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds

scores.append(accuracy_score(y_test.values, preds))
print(scores[-1])

sess.close()

print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

epoch 0; iter: 0; batch classifier loss: 109.013351; batch adversarial loss: 0.829496
epoch 1; iter: 0; batch classifier loss: 64.578354; batch adversarial loss: 0.860782
epoch 2; iter: 0; batch classifier loss: 94.886574; batch adversarial loss: 0.844943
epoch 3; iter: 0; batch classifier loss: 85.343430; batch adversarial loss: 0.822847
epoch 4; iter: 0; batch classifier loss: 101.623352; batch adversarial loss: 0.835647
epoch 5; iter: 0; batch classifier loss: 91.026535; batch adversarial loss: 0.846654
epoch 6; iter: 0; batch classifier loss: 71.284225; batch adversarial loss: 0.889858
epoch 7; iter: 0; batch classifier loss: 89.164337; batch adversarial loss: 0.846509
epoch 8; iter: 0; batch classifier loss: 73.869446; batch adversarial loss: 0.817984
epoch 9; iter: 0; batch classifier loss: 59.876816; batch adversarial loss: 0.853224
epoch 10; iter: 0; batch classifier loss: 54.859310; batch adversarial loss: 0.824510
epoch 11; iter: 0; batch classifier loss: 69.370575; batch adv

In [97]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = y_pred_4

        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.011701170117011772
Max Average Odds Difference (Ideal value = 0): 0.03238428238428238
Max Equal Opportunity Difference (Ideal value = 0): 0.16666666666666666
Max Theil Index (Ideal value = 0): 0.2901918310515196
Max Generalized Entropy Index (Ideal value = 0): 0.1888


### AD + CEOP

In [126]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []


# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Loan_Status', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[1]])

# Apply the AdversarialDebiasing algorithm
tf.reset_default_graph()
sess = tf.Session()
adv_debiasing = AdversarialDebiasing(privileged_groups=priv_group,
                                     unprivileged_groups=unprivileged_groups,
                                     scope_name='adv_debiasing',
                                     debias=True,
                                     sess=sess)
adv_debiasing.fit(aif360_train)

# Transform the training data
aif360_train_pred = adv_debiasing.predict(aif360_train)
X_Train_transformed = aif360_train_pred.features
y_Train_transformed = aif360_train_pred.labels.ravel()

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_Train_transformed, y_Train_transformed)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds

aif360_test = StandardDataset(pd.concat([X_test, pd.DataFrame(y_test, columns=['Loan_Status'], index=X_test.index)], axis=1),
                                      label_name='Loan_Status', favorable_classes=[1],
                                      protected_attribute_names=['Gender'],
                                      privileged_classes=[[1]])
aif360_preds = StandardDataset(pd.concat([X_test,pd.DataFrame(y_pred_4, columns=['Loan_Status'], index=X_test.index)], axis=1),
                               label_name='Loan_Status', favorable_classes=[1],
                               protected_attribute_names=['Gender'],
                               privileged_classes=[[1]])

eq_odds = CalibratedEqOddsPostprocessing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}], seed=42)
eq_odds = eq_odds.fit(aif360_test, aif360_preds)
transformed_preds = eq_odds.predict(aif360_preds).labels

scores.append(accuracy_score(y_test.values, transformed_preds))
print(scores[-1])

sess.close()

print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

epoch 0; iter: 0; batch classifier loss: 131.230621; batch adversarial loss: 0.651781
epoch 1; iter: 0; batch classifier loss: 248.220840; batch adversarial loss: 0.649343
epoch 2; iter: 0; batch classifier loss: 195.978348; batch adversarial loss: 0.635670
epoch 3; iter: 0; batch classifier loss: 165.668884; batch adversarial loss: 0.659291
epoch 4; iter: 0; batch classifier loss: 187.394226; batch adversarial loss: 0.662505
epoch 5; iter: 0; batch classifier loss: 92.589355; batch adversarial loss: 0.649533
epoch 6; iter: 0; batch classifier loss: 93.514832; batch adversarial loss: 0.651316
epoch 7; iter: 0; batch classifier loss: 94.634262; batch adversarial loss: 0.654391
epoch 8; iter: 0; batch classifier loss: 98.928352; batch adversarial loss: 0.660300
epoch 9; iter: 0; batch classifier loss: 101.683533; batch adversarial loss: 0.639639
epoch 10; iter: 0; batch classifier loss: 99.686401; batch adversarial loss: 0.640373
epoch 11; iter: 0; batch classifier loss: 140.755585; batc

In [127]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = transformed_preds

        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.045454545454545456
Max Average Odds Difference (Ideal value = 0): 0.038461538461538464
Max Equal Opportunity Difference (Ideal value = 0): 0.07692307692307687
Max Theil Index (Ideal value = 0): 0.3165588528239179
Max Generalized Entropy Index (Ideal value = 0): 0.19067745441371797


## RW + CEOP

In [128]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []

# for train_index, test_index in StratifiedKFold(n_splits=5).split(X.values, y.values):
#     X_Train, X_Test = X.iloc[train_index, :], X.iloc[test_index, :]
#     y_Train, y_Test = y.iloc[train_index], y.iloc[test_index]

# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Loan_Status', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[1]])
weights = aif360_train.instance_weights
preprocessor = Reweighing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}])
train_dataset_reweighted = preprocessor.fit_transform(aif360_train)
X_Train_transformed = train_dataset_reweighted.features
y_Train_transformed = train_dataset_reweighted.labels.ravel()

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_Train_transformed, y_Train_transformed, sample_weight=weights)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds

aif360_test = StandardDataset(pd.concat([X_test, pd.DataFrame(y_test, columns=['Loan_Status'], index=X_test.index)], axis=1),
                                      label_name='Loan_Status', favorable_classes=[1],
                                      protected_attribute_names=['Gender'],
                                      privileged_classes=[[1]])
aif360_preds = StandardDataset(pd.concat([X_test,pd.DataFrame(y_pred_4, columns=['Loan_Status'], index=X_test.index)], axis=1),
                               label_name='Loan_Status', favorable_classes=[1],
                               protected_attribute_names=['Gender'],
                               privileged_classes=[[1]])

eq_odds = CalibratedEqOddsPostprocessing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}], seed=42)
eq_odds = eq_odds.fit(aif360_test, aif360_preds)
transformed_preds = eq_odds.predict(aif360_preds).labels



scores.append(accuracy_score(y_test.values, transformed_preds))
print(scores[-1])

sess.close()

print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

0.7642276422764228
Mean: 0.7642276422764228 
STD:  0.0 



In [129]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = transformed_preds

        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.1539153915391539
Max Average Odds Difference (Ideal value = 0): 0.1522782772782773
Max Equal Opportunity Difference (Ideal value = 0): 0.3055555555555556
Max Theil Index (Ideal value = 0): 0.2199055405430055
Max Generalized Entropy Index (Ideal value = 0): 0.14326131687242796
